In [12]:
import pandas as pd
import numpy as np
import json
import os
# import time
# from memory_profiler import memory_usage

# display all the  columns
pd.set_option('display.max_columns', None)
#최대표시 컬럼 수 
pd.options.display.max_columns=100
#최대표시 row 수
pd.options.display.max_rows = 999
#print 함수 없이 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
#주피터 화면 확장
from IPython.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>"))

In [13]:
os.getcwd()

'c:\\dev\\workspace\\data_analyzer\\notebook'

In [14]:
df = pd.read_csv("../data/kaggle_ecommerce_behavior_data_2019_Nov_partial.csv")
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:10 UTC,view,1005115,2053013555631882655,electronics.smartphone,apple,949.47,562079864,0fe64462-83eb-4131-ab00-8e764dd995c3
1,2019-11-01 00:00:19 UTC,view,25600085,2053013559675191951,NaN,brw,134.37,522031876,39d48518-9fca-4df3-9724-950cd6ec44eb
2,2019-11-01 00:02:53 UTC,view,40700033,2086471225382536008,NaN,komfort-s,43.73,522031876,39d48518-9fca-4df3-9724-950cd6ec44eb
3,2019-11-01 00:03:11 UTC,view,40700034,2086471225382536008,NaN,komfort-s,32.15,522031876,39d48518-9fca-4df3-9724-950cd6ec44eb
4,2019-11-01 00:03:23 UTC,view,40700034,2086471225382536008,NaN,komfort-s,32.15,522031876,39d48518-9fca-4df3-9724-950cd6ec44eb
...,...,...,...,...,...,...,...,...,...
912807,2019-11-30 23:50:54 UTC,view,2702309,2053013563911439225,appliances.kitchen.refrigerators,dauscher,256.89,513669664,8aed6cef-2501-4263-a2cd-ddce0ef24ccd
912808,2019-11-30 23:51:44 UTC,view,2702309,2053013563911439225,appliances.kitchen.refrigerators,dauscher,256.89,513669664,8aed6cef-2501-4263-a2cd-ddce0ef24ccd
912809,2019-11-30 23:52:15 UTC,view,2700306,2053013563911439225,appliances.kitchen.refrigerators,NaN,269.48,513669664,8aed6cef-2501-4263-a2cd-ddce0ef24ccd
912810,2019-11-30 23:52:42 UTC,view,2701658,2053013563911439225,appliances.kitchen.refrigerators,indesit,272.54,513669664,8aed6cef-2501-4263-a2cd-ddce0ef24ccd


# 정합성 체크

In [157]:
def check_data_consistency(params, data):
    info = params["preprocess"]["consistency"]
    abnormal_cases = dict()
    for key in info:
        df_groupby = data.groupby(key)[info[key]].nunique()
        df_groupby["sum"] = df_groupby.sum(axis=1)
        abnormal_cases[key] = df_groupby.loc[
            df_groupby["sum"] > len(info[key])
        ].index.tolist()
    return abnormal_cases

def generate_convert_info(params, data, abnormal_cases):
    infos = dict()
    for key in abnormal_cases:
        if abnormal_cases[key]:
            related_keys = params["preprocess"]["consistency"][key]
            n_data = data.loc[data[key].isin(abnormal_cases[key])]
            infos[key] = n_data.groupby(key).agg({rkey: "unique" for rkey in related_keys}).to_dict("index")
    return infos

def load_json(dir):
    with open(dir, encoding="utf-8-sig") as f:
        json_file = json.load(f)
    return json_file


# dfa.groupby("product_id").agg({"category_id":"unique", "brand":"unique"}).to_dict("index")


In [158]:
params = load_json("../params/param_test.json")
params

res_dict = check_data_consistency(params, df)
res_dict

{'preprocess': {'consistency': {'user_session': ['user_id'],
   'category_id': ['category_code'],
   'product_id': ['category_id', 'brand']}}}

{'user_session': [],
 'category_id': [],
 'product_id': [1701027,
  2702677,
  10701166,
  12000324,
  16600184,
  17200973,
  26800016,
  26900064,
  28101062,
  57600030]}

In [159]:
generate_convert_info(params, df, res_dict)

{'product_id': {1701027: {'category_id': array([2053013553031414015], dtype=int64),
   'brand': array(['samsung', 'lenovo'], dtype=object)},
  2702677: {'category_id': array([2053013563911439225], dtype=int64),
   'brand': array(['indesit', 'schaublorenz'], dtype=object)},
  10701166: {'category_id': array([2053013561277416167], dtype=int64),
   'brand': array(['sonyinteractiveentertainment', 'badlandgames'], dtype=object)},
  12000324: {'category_id': array([2053013555438944659], dtype=int64),
   'brand': array(['caramelia', 'tatami'], dtype=object)},
  16600184: {'category_id': array([2053013560287560373], dtype=int64),
   'brand': array(['bugati', 'bugatti'], dtype=object)},
  17200973: {'category_id': array([2053013559792632471], dtype=int64),
   'brand': array(['mebberykz', 'mebberi'], dtype=object)},
  26800016: {'category_id': array([2053013564641248165], dtype=int64),
   'brand': array(['fitlux', 'jkexer'], dtype=object)},
  26900064: {'category_id': array([2053013564599305123]